# Recon2.2plusthreeCompetition.json [Transport Competitive mechanism only]

##Changing medium

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


# Other treatment

## Exercise (Other treatment)

In [3]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
model=M.copy()

In [4]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [5]:
# Changing the phe and tyr in the medium
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.04

In [6]:
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# Phe secretion in the blood is blocked, otherwise there is no competition.
model.reactions.EX_phe_L_BD.upper_bound=0
# Phenylpyruvate production in the brain and Liver are limited
model.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model.reactions.get_by_id('PHPYRte').lower_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti1_To1.upper_bound=0.198
### Only consider the transport competitive mechanism
model.reactions.Phe_TH_INB.upper_bound=0
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [7]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model1.reactions.get_by_id('PHPYRte').lower_bound=-0.1
sol = model1.optimize()
sol.objective_value

0.18734518083433527

In [8]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [9]:
#Remembering to add other amino acid
model1.reactions.EX_glc_e.lower_bound=-2##Do not need much glucsoe.
model1.reactions.EX_his_L_e.lower_bound=-0.2
model1.reactions.EX_ile_L_e.lower_bound=-0.2
model1.reactions.EX_leu_L_e.lower_bound=-0.2
model1.reactions.EX_lys_L_e.lower_bound=-0.2
model1.reactions.EX_met_L_e.lower_bound=-0.2
model1.reactions.EX_thr_L_e.lower_bound=-0.2
model1.reactions.EX_trp_L_e.lower_bound=-0.1
model1.reactions.EX_val_L_e.lower_bound=-0.2
model1.reactions.EX_tyr_L_e.lower_bound=-0.1

In [10]:
model1.reactions.biomass_reaction.upper_bound=0.5
sol = model1.optimize()
sol.objective_value

0.24190296608223907

In [11]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

## More carriers for model

In [12]:
model2=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model2.genes.get_by_id('HGNC:8582').knock_out()
model2.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model2.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model2.reactions.Ti1_To1.upper_bound=0.3
sol = model2.optimize()
sol.objective_value
b = show_map(sol,map_loc)
b

0.2140536

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

# Young PKU patient VS elderly PKU patient with Recon2.2plusthreeCompetitive

##We assume the treatment should not produce the Phenylpyruvate in the system.

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
model=M.copy()

In [4]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [5]:
# Changing the phe and tyr in the medium
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [6]:
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# Phe secretion in the blood is blocked, otherwise there is no competition.
model.reactions.EX_phe_L_BD.upper_bound=0
# Phenylpyruvate production in the brain and Liver are limited
model.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model.reactions.get_by_id('PHPYRte').lower_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti1_To1.upper_bound=0.198
### Only consider the transport competitive mechanism
model.reactions.Phe_TH_INB.upper_bound=0
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554000000001

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [7]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model1.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model1.reactions.get_by_id('PHPYRte').lower_bound=-0.1
sol = model1.optimize()
sol.objective_value

0.18734518083433555

In [8]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [52]:
# For children
model2=model1.copy()
model2.solver='glpk'

In [53]:
model2.reactions.biomass_reaction_INB.upper_bound=0.1
model2.reactions.biomass_reaction_INB.lower_bound=0.1
model2.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.0596
model2.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.0596
model2.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model2.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No Phenylpyruvate production in liver
model2.reactions.PHETA1.upper_bound=0.0001
model2.reactions.EX_phe_L_e.upper_bound=-0.039
model2.reactions.EX_phe_L_e.lower_bound=-0.039
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

##For children, the basic need for Phe is 0.0389. And if they need the biomass, dopamine and serotonin in the brain be normal, they should decrease the Phe uptake flux to 0.04.

In [56]:
#For adult person
model3=model1.copy()
### No Phenylpyruvate production in liver
model3.reactions.PHETA1.upper_bound=0.0001
model3.reactions.biomass_reaction_INB.upper_bound=0.05
model3.reactions.biomass_reaction_INB.lower_bound=0.05
model3.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.0596
model3.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.0596
model3.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model3.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
model3.reactions.EX_phe_L_e.upper_bound=-0.0475
model3.reactions.EX_phe_L_e.lower_bound=-0.0475
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [58]:
#For old person
model4=model1.copy()
### No Phenylpyruvate production in liver
model4.reactions.PHETA1.upper_bound=0.0001
model4.reactions.biomass_reaction_INB.upper_bound=0.02
model4.reactions.biomass_reaction_INB.lower_bound=0.02
model4.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.07
model4.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.07
model4.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model4.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
model4.reactions.EX_phe_L_e.upper_bound=-0.0425
model4.reactions.EX_phe_L_e.lower_bound=-0.0425
fbasol = model4.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [62]:
#For old person (less biomass and more dopamine)
model5=model1.copy()
### No Phenylpyruvate production in liver
model5.reactions.PHETA1.upper_bound=0.0001
model5.reactions.biomass_reaction_INB.upper_bound=0.02
model5.reactions.biomass_reaction_INB.lower_bound=0.02
model5.reactions.get_by_id('3HLYTCL_INB').upper_bound=0.06
model5.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.06
model5.reactions.get_by_id('5HLTDL_INB').upper_bound=0.095
model5.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
model5.reactions.EX_phe_L_e.upper_bound=-0.0525
model5.reactions.EX_phe_L_e.lower_bound=-0.0525
fbasol = model5.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…